In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import sys
sys.path.append('/content/drive/MyDrive/SSD_detection')

In [5]:
import time
import torch.backends.cudnn as cudnn
import torch.optim
from model import SSD300, MultiBoxLoss
import torch.utils.data
from utils import *
from datasets import CustomDataset

21
label map: {'mango': 1, 'melon': 2, 'strawberry': 3, 'blueberry': 4, 'apple': 5, 'grapefruit': 6, 'plum': 7, 'peach': 8, 'grape': 9, 'cherry': 10, 'yam': 11, 'chestnuts': 12, 'pepper': 13, 'chicory': 14, 'Kohlrabi': 15, 'Paprika': 16, 'Tomato': 17, 'Mushroom': 18, 'Pumpkin': 19, 'Pimento': 20, 'background': 0}
label color map {'mango': '#e6194b', 'melon': '#3cb44b', 'strawberry': '#ffe119', 'blueberry': '#0082c8', 'apple': '#f58231', 'grapefruit': '#911eb4', 'plum': '#46f0f0', 'peach': '#f032e6', 'grape': '#d2f53c', 'cherry': '#fabebe', 'yam': '#008080', 'chestnuts': '#000080', 'pepper': '#aa6e28', 'chicory': '#fffac8', 'Kohlrabi': '#800000', 'Paprika': '#aaffc3', 'Tomato': '#808000', 'Mushroom': '#ffd8b1', 'Pumpkin': '#e6beff', 'Pimento': '#808080', 'background': '#FFFFFF'}
reverse label map {1: 'mango', 2: 'melon', 3: 'strawberry', 4: 'blueberry', 5: 'apple', 6: 'grapefruit', 7: 'plum', 8: 'peach', 9: 'grape', 10: 'cherry', 11: 'yam', 12: 'chestnuts', 13: 'pepper', 14: 'chicory', 

In [6]:
data_folder = '/content/drive/MyDrive/SSD_detection/train'

In [7]:
global start_epoch, label_map, epoch, checkpoint, decay_lr_at

In [8]:
# Model parameters
n_classes = len(label_map)
print(n_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Learning parameters
checkpoint = None  # train mode on/off
batch_size = 4
iterations = 3
# workers = 4
print_freq = 100
lr = 1e-3
decay_lr_at = [80000, 100000]
decay_lr_to = 0.1  # decay learning rate to this fraction of the existing learning rate
momentum = 0.9  # momentum
weight_decay = 5e-4  # weight decay
grad_clip = None
cudnn.benchmark = True

21
cuda


In [9]:
def train(train_loader, model, criterion, optimizer, epoch):
    model.train()  # training mode enables dropout

    batch_time = AverageMeter()  # forward prop. + back prop. time
    data_time = AverageMeter()  # data loading time
    losses = AverageMeter()  # loss

    start = time.time()

    # Batches
    for i, (images, boxes, labels) in enumerate(train_loader):
        # images: (N, 3, 300, 300)
        data_time.update(time.time() - start)

        # Move to default device
        # images
        images = images.to(device)  # (batch_size (N), 3, 300, 300)
        boxes = [b.to(device) for b in boxes]
        labels = [l.to(device) for l in labels]

        # Forward prop.
        # 여기서 model.py의 forward 함수의 인자로 넣어줄 images 가 전달된다.
        predicted_locs, predicted_scores = model(images)  # (N, 8732, 4), (N, 8732, n_classes)

        # Loss

        loss = criterion(predicted_locs, predicted_scores, boxes, labels)  # scalar

        # Backward prop.
        optimizer.zero_grad()
        loss.backward()

        # Clip gradients, if necessary
        if grad_clip is not None:
            clip_gradient(optimizer, grad_clip)

        # Update model
        optimizer.step()

        losses.update(loss.item(), images.size(0))
        batch_time.update(time.time() - start)

        start = time.time()

        # Print status
        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Batch Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data Time {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(epoch, i, len(train_loader),
                                                                  batch_time=batch_time,
                                                                  data_time=data_time, loss=losses))
    del predicted_locs, predicted_scores, images, boxes, labels  # free some memory since their histories may be stored

In [10]:
if checkpoint is None:
    start_epoch = 0
    model = SSD300(n_classes=21)
    biases = list()
    not_biases = list()
    for param_name, param in model.named_parameters():
        if param.requires_grad:
            if param_name.endswith('.bias'):
                biases.append(param)
            else:
                not_biases.append(param)
    optimizer = torch.optim.SGD(params=[{'params': biases, 'lr': 2 * lr}, {'params': not_biases}],
                                lr=lr, momentum=momentum, weight_decay=weight_decay)

else:
    checkpoint = torch.load(checkpoint)
    start_epoch = checkpoint['epoch'] + 1
    print('\nLoaded checkpoint from epoch %d.\n' % start_epoch)
    model = checkpoint['model']
    optimizer = checkpoint['optimizer']

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 99.5MB/s]


In [11]:
# Move to default device
model = model.to(device)

# loss 함수 지정
criterion = MultiBoxLoss(priors_cxcy=model.priors_cxcy).to(device)

# Custom dataloaders
train_dataset = CustomDataset(data_folder, split='train')
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                           collate_fn=train_dataset.collate_fn,
                                           pin_memory=True)  # note that we're passing the collate function here

In [12]:
epochs = 20
# decay_lr_at = [it // (len(train_dataset) // 32) for it in decay_lr_at]
decay_lr_at = [10,18]
print(epochs)
print(decay_lr_at)

20
[10, 18]


In [13]:
# Epochs
for epoch in range(start_epoch, epochs):
    # Decay learning rate at particular epochs
    if epoch in decay_lr_at:
        adjust_learning_rate(optimizer, decay_lr_to)

    # One epoch's train, train 함수로 학습 진행
    train(train_loader=train_loader, model=model, criterion=criterion,
          optimizer=optimizer,
          epoch=epoch)
    # Save checkpoint
    save_checkpoint(epoch, model, optimizer)

Epoch: [0][0/2000]	Batch Time 60.981 (60.981)	Data Time 46.717 (46.717)	Loss 25.6558 (25.6558)	
Epoch: [0][100/2000]	Batch Time 2.690 (3.427)	Data Time 2.453 (3.040)	Loss 10.9986 (13.6168)	
Epoch: [0][200/2000]	Batch Time 2.811 (3.136)	Data Time 2.558 (2.820)	Loss 7.2599 (10.5656)	
Epoch: [0][300/2000]	Batch Time 2.818 (3.040)	Data Time 2.588 (2.748)	Loss 5.0437 (9.2567)	
Epoch: [0][400/2000]	Batch Time 2.795 (2.987)	Data Time 2.538 (2.707)	Loss 5.4730 (8.5539)	
Epoch: [0][500/2000]	Batch Time 2.802 (2.965)	Data Time 2.553 (2.691)	Loss 6.2776 (8.0965)	
Epoch: [0][600/2000]	Batch Time 2.803 (2.943)	Data Time 2.536 (2.674)	Loss 5.4654 (7.7596)	
Epoch: [0][700/2000]	Batch Time 2.856 (2.930)	Data Time 2.626 (2.664)	Loss 5.4174 (7.5447)	
Epoch: [0][800/2000]	Batch Time 2.729 (2.920)	Data Time 2.493 (2.657)	Loss 5.3711 (7.3679)	
Epoch: [0][900/2000]	Batch Time 3.025 (2.916)	Data Time 2.793 (2.655)	Loss 6.4175 (7.2200)	
Epoch: [0][1000/2000]	Batch Time 2.814 (2.910)	Data Time 2.573 (2.650)	Lo

FileNotFoundError: ignored